In [7]:
%cd /home/thucth/thucth/project/id_occlusion

/home/thucth/thucth/project/id_occlusion


## Prepare dataset

```
Create csv file with path _ label(float score)

Description:
- Dataset with multi level:
0: no occlusion (or occluded area is out of id)
1: 10% occlusion
2: 20% occlusion
3: 30% occlusion
4: 40% occlusion
5: 50% occlusion
.
.
```

### Inherit data from IQ006

In [1]:
import pandas as pd
import json 
categories_new = [
    {"id": 1, "name": 'corner_1'},
    {"id": 2, "name": 'corner_2'},
    {"id": 3, "name": 'corner_3'},
    {"id": 4, "name": 'corner_4'},
    {"id": 5, "name": 'full'}
]

categories_map = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    'full': 5
}

categories_map_inverse = {
    1: "1",
    2: "2",
    3: "3",
    4: "4",
    5: "full"
}


In [27]:
import json
def read_from_cocojson(path_json):
    images_dict={}
    with open(path_json, 'r') as f:
        data = json.load(f)
        for ann in data['annotations']:
            if ann["image_id"] not in images_dict:
                images_dict[ann["image_id"]]={"bbox":{}}
            corner_type = categories_map_inverse[ann["category_id"]]
            #this is to convert from coco format into voc format
            images_dict[ann["image_id"]]["bbox"][corner_type] = [ann["bbox"][0], ann["bbox"][1], ann["bbox"][2]+ann["bbox"][0], ann["bbox"][3]+ann["bbox"][1]]

        for img in data['images']:
            images_dict[img['id']]["path"]= img["file_name"]
            images_dict[img['id']]["width"] = img["width"]
            images_dict[img['id']]["height"] = img["height"]
    voc_data = [images_dict[k] for k in images_dict.keys()]
    return voc_data

### generate clear id

In [17]:
import csv
def generate_data_clear(voc_data, path_csv):
    print("Amount of source: ", len(voc_data))
    count=0
    f = open(path_csv, 'w')
    csv_writer = csv.writer(f)
    csv_writer.writerow(['path','score'])
    for sample in voc_data:
        if  len(sample['bbox'].keys())==5: #full corner
            csv_writer.writerow([sample['path'], 0])
            count+=1
    print("Amount of target: ", count)
    f.close()

split='val'
voc_data = read_from_cocojson(path_json = "/home/thucth/thucth/project/id_fullcorner_dataprocess/json_data/original_size_%s_2702.json"%split)
generate_data_clear(voc_data, "resources/data_occlusion_%s.csv"%split)

### generate occluded id

In [26]:
from src.datasets import augment
import cv2
import importlib
importlib.reload(augment)


foregrounds = augment.load_foreground(path="./resources/foreground")


Amount of source:  950
Amount of target:  858
